In [ ]:
import numpy as np
from PIL import Image

In [31]:
def rgb_to_gray(image_path):
    img = Image.open(image_path)
    img_array = np.array(img)

    grayscale = 0.2989 * img_array[:, :, 0] + 0.5870 * img_array[:, :, 1] + 0.1140 * img_array[:, :, 2]

    return grayscale.astype('uint8'), img_array

def method_otsu(gray_image):
    total_pixels = gray_image.size
    pixel_counts = np.bincount(gray_image.flatten(), minlength=256)

    sum_total = np.dot(np.arange(256), pixel_counts)
    weight_background = 0
    sum_background = 0
    max_variance = 0
    threshold = 0

    for t in range(256):
        weight_background += pixel_counts[t]
        if weight_background == 0:
            continue
        
        weight_foreground = total_pixels - weight_background
        if weight_foreground == 0:
            break
        
        sum_background += t * pixel_counts[t]
        sum_foreground = sum_total - sum_background

        mean_background = sum_background / weight_background
        mean_foreground = sum_foreground / weight_foreground

        variance_between = weight_background * weight_foreground * (mean_background - mean_foreground) ** 2

        if variance_between > max_variance:
            max_variance = variance_between
            threshold = t

    binary_image = np.where(gray_image > threshold, 0, 255)
    return binary_image.astype('uint8'), threshold

def extract_object(original_image, mask):
    # Створюємо логічну маску з бінарного зображення
    mask = mask == 255
    # Створюємо порожнє зображення з тим самим розміром, що і оригінал
    extracted_object = np.zeros_like(original_image)
    # Копіюємо з оригіналу тільки ті пікселі, де маска істина
    extracted_object[mask] = original_image[mask]

    return extracted_object.astype('uint8')

# Основна функція
def main(image_path):
    gray_image, original_image = rgb_to_gray(image_path)
    binary_image, threshold = method_otsu(gray_image)
    extracted_object = extract_object(original_image, binary_image)

    # Збереження результатів
    Image.fromarray(gray_image).save('grayscale_image.png')
    Image.fromarray(binary_image).save('binary_image.png')
    Image.fromarray(extracted_object).save('extracted_object.png') 

In [32]:
main('C:/Users/Стелла/Desktop/CompVision/Lab1/Lab1/photo3.jpg')